In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.runconfig import RunConfiguration, MpiConfiguration
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='benchy', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters_benchmarking')

In [9]:
if 'weather-files' not in ws.datasets:
    ds = Dataset.File.from_files('https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather/*/*/*.parquet', validate=False)
    ds = ds.register(ws, 'weather-files')
else:
    ds = ws.datasets['weather-files']
    
ds

2019-12-21 17:40:55.565814 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=5760.43 [ms], Info = {'activity_id': 'fb15e69d-2f4b-4d53-a4fe-449e2ec41507', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.0.79', 'completionStatus': 'Success', 'durationMs': 0.12}, Exception=Exception; Cannot find dataset registered with name "weather-files" in the workspace.
2019-12-21 17:41:01.327192 | ActivityCompleted: Activity=from_files, HowEnded=Failure, Duration=3.65 [ms], Info = {'activity_id': '750cdcd6-ca74-4ddf-8a85-89710c9a36ea', 'activity_name': 'from_files', 'activity_type': 'PublicApi', 'app_name': 'FileDataset', 'source': 'azureml.dataset', 'version': '1.0.79', 'completionStatus': 'Failure', 'durationMs': 5760.43}, Exception=ImportError; Could not import package "azureml-dataprep". Please ensure it is installed by running: pip install "azureml-dataprep[fuse,pandas]"


ImportError: Could not import package "azureml-dataprep". Please ensure it is installed by running: pip install "azureml-dataprep[fuse,pandas]"

In [ ]:
!pip freeze | grep dataprep

In [8]:
import azureml.dataprep as dprep

ImportError: cannot import name 'runtime' from 'dotnetcore2' (unknown location)